Now here we defin the all the transforms that we will be using in these model las 


<!-- Here we Define our model las -->


In [4]:
class ShuffleNet(torch.nn.Module):
    def __init__(self, in_channels=3, out_channels=64, stride=1, groups=1):
        super(ShuffleNet, self).__init__()

        self.groups = groups

        #  group convolution

        self.gconv1 = torch.nn.Conv2d(in_channels, out_channels, 1, groups=groups, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(out_channels)
        self.relu1 = torch.nn.ReLU()

        
        #  depthwise convolution
        

        self.dwconv = torch.nn.Conv2d(out_channels, out_channels, 3, stride=stride, padding=1, groups=out_channels, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(out_channels)
        self.relu2 = torch.nn.ReLU()

        #  group convolution

        self.gconv2 = torch.nn.Conv2d(out_channels, out_channels, 1, groups=groups, bias=False)
        self.bn3 = torch.nn.BatchNorm2d(out_channels)

        # fc

        self.fc = torch.nn.Linear(out_channels, 2)


    def forward(self, x):
        out = self.gconv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.dwconv(out)
        out = self.bn2(out)
        out = self.relu2(out)

        out = self.gconv2(out)
        out = self.bn3(out)
        out = self.fc(out)

        
        return out


In [1]:
#

In [7]:
# load  pre trained yolo detector model and train classifier

import torch
import torch.nn as nn
from models.yolo import Model  # Import the YOLOv5 model class


detector_model = torch.hub.load('.', 'custom', path='/home/purushotham/Downloads/smok.pt', source='local',force_reload=True) 

detector_model.load_state_dict(detector_model.state_dict())


# Set the model to evaluation mode
detector_model.eval()


class Yolo(nn.Module):
    def __init__(self):
        super(Yolo, self).__init__()
        
        for i in detector_model.children():
            self.backbone = nn.Sequential(*list(i.model.model[:13])) 
            break
#         self.backbone = nn.Sequential(*list(detector_model.children().model.model[:13])) 
        
#         self.backbone = nn.Sequential(*list(detector_model.model.model[:13]))
        self.linear = nn.Linear(192, 512)
        self.relu = nn.ReLU()
        self.linear2=torch.nn.Linear(512, 2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        x = self.backbone(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x
    
    
    

     break
    

YOLOv5 🚀 v6.2-157-g30fa9b61 Python-3.8.10 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5947MiB)

Fusing layers... 
Model summary: 280 layers, 3093052 parameters, 0 gradients, 4.2 GFLOPs
Adding AutoShape... 


In [8]:
 class ShuffleNet(torch.nn.Module):
    def __init__(self):
        super(ShuffleNet, self).__init__()

        # Define the model layers
        self.conv1 = torch.nn.Conv2d(3, 32, 3)
        self.bn1 = torch.nn.BatchNorm2d(32)
        self.relu1 = torch.nn.ReLU()

        self.conv2 = torch.nn.Conv2d(32, 64, 3)
        self.bn2 = torch.nn.BatchNorm2d(64)
        self.relu2 = torch.nn.ReLU()

        self.pool = torch.nn.MaxPool2d(2, 2)

        self.fc1 = torch.nn.Linear(64 * 112 * 112, 1024)
        self.fc2 = torch.nn.Linear(1024, 2)

    def forward(self, x):
        # Forward pass the input through the model layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.pool(x)

        x = x.view(-1, 64 * 112 * 112)

        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)

        return x


In [9]:
#encoders and decoders

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
class BottleneckNet(nn.Module):
    def __init__(self):
        super(BottleneckNet, self).__init__()
        self.encoder = nn.Sequential(nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
        )
        self.bottleneck = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, 2),
            nn.Sigmoid(),
        )

#     def forward(self, x):
#         x = self.encoder(x)
#         x = self.bottleneck(x)
#         x = self.decoder(x)
#         return x

In [10]:
# simple small architecture
class SmallNet(nn.Module):
    def __init__(self):
        super(Net, self).__init()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64  ,5  ,5, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64  5  5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        
         #convultion layers
            
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2,stride=2)
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2,stride=2),
        )

          #fc layers
        self.classifier = nn.Sequential(
            nn.Linear(400,120),  
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10),
            nn.Softmax()

        )
        
    def forward(self,x): 
        y=self.feature_extractor(x)
        print(y.shape)
        output= torch.flatten(y,1)
        output=self.classifier(y)
        return ouput

In [12]:
class AlexNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        
         #convultion layers
            
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),  
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, 5, padding=2),  
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2), 
            nn.Conv2d(256, 384, 3, padding=1),  
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2), 
        )

          #fc layers
        self.classifier = nn.Sequential(
            nn.Linear(4096,4096),  
            nn.ReLU(),
            nn.Linear(4096,2),
            nn.Softmax()

        )
        
    def forward(self,x): 
        y=self.feature_extractor(x)
        print(y.shape)
        output= torch.flatten(y,1)
        output=self.classifier(y)
        return ouput

In [13]:
class VGG(nn.Module):
    """
    Standard PyTorch implementation of VGG. Pretrained imagenet model is used.
    """
    def __init__(self):
        super().__init__()
    

            # conv1
        self.conv1=nn.Sequential( nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True))
            
            # conv2
        self.conv2=nn.Sequential(nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True))

           self.conv3=nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True))

            self.conv4=nn.Sequential(
            nn.Conv2d(256, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True))

            self.conv5=nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2, return_indices=True))
        

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 1000),
             nn.Softmax()
        )

    
        
    def forward(self, x):
        
    
    
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
       
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x

Now Testing the model las 


Now We will change the above infrenced model into the tensorrt engine las

